# How to add summary of the conversation history

One of the most common use cases for persistence is to use it to keep track of conversation history. This is great - it makes it easy to continue conversations. As conversations get longer and longer, however, this conversation history can build up and take up more and more of the context window. This can often be undesirable as it leads to more expensive and longer calls to the LLM, and potentially ones that error. One way to work around that is to create a summary of the conversation to date, and use that with the past N messages. This guide will go through an example of how to do that.

This will involve a few steps:

- Check if the conversation is too long (can be done by checking number of messages or length of messages)
- If yes, the create summary (will need a prompt for this)
- Then remove all except the last N messages

A big part of this is deleting old messages. For an in depth guide on how to do that, see [this guide](../delete-messages)

## Setup

First, let's set up the packages we're going to want to use

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_anthropic

Next, we need to set API keys for Anthropic (the LLM we will use)

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")

ANTHROPIC_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Build the chatbot

Let's now build the chatbot.

In [3]:
from typing import Literal

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, RemoveMessage
from langgraph.checkpoint.redis import RedisSaver
from langgraph.graph import MessagesState, StateGraph, START, END

# Set up Redis connection for checkpointer
REDIS_URI = "redis://redis:6379"
memory = None
with RedisSaver.from_conn_string(REDIS_URI) as cp:
    cp.setup()
    memory = cp


# We will add a `summary` attribute (in addition to `messages` key,
# which MessagesState already has)
class State(MessagesState):
    summary: str


# We will use this model for both the conversation and the summarization
model = ChatAnthropic(model_name="claude-3-haiku-20240307")


# Define the logic to call the model
def call_model(state: State):
    # If a summary exists, we add this in as a system message
    summary = state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# We now define the logic for determining whether to end or summarize the conversation
def should_continue(state: State) -> Literal["summarize_conversation", END]:
    """Return the next node to execute."""
    messages = state["messages"]
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    # Otherwise we can just end
    return END


def summarize_conversation(state: State):
    # First, we summarize the conversation
    summary = state.get("summary", "")
    if summary:
        # If a summary already exists, we use a different system prompt
        # to summarize it than if one didn't
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    # We now need to delete messages that we no longer want to show up
    # I will delete all but the last two messages, but you can change this
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}


# Define a new graph
workflow = StateGraph(State)

# Define the conversation node and the summarize node
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `conversation`.
    # This means these are the edges taken after the `conversation` node is called.
    "conversation",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `summarize_conversation` to END.
# This means that after `summarize_conversation` is called, we end.
workflow.add_edge("summarize_conversation", END)

# Finally, we compile it!
app = workflow.compile(checkpointer=memory)

19:51:26 langgraph.checkpoint.redis INFO   Redis client is a standalone client
19:51:26 redisvl.index.index INFO   Index already exists, not overwriting.
19:51:26 redisvl.index.index INFO   Index already exists, not overwriting.
19:51:26 redisvl.index.index INFO   Index already exists, not overwriting.


## Using the graph

In [4]:
def print_update(update):
    for k, v in update.items():
        for m in v["messages"]:
            m.pretty_print()
        if "summary" in v:
            print(v["summary"])

In [5]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "4"}}
input_message = HumanMessage(content="hi! I'm bob")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="i like the celtics!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

hi! I'm bob
19:51:27 httpx INFO   HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
================================== Ai Message ==================================

Hi Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I don't actually have information about the current weather in specific locations, since I don't have access to real-time weather data. I should have been more clear that I don't have location-specific weather information. Please let me know if there's anything else I can assist you with!
================================ Human Message =================================

what's my name?
19:51:28 httpx INFO   HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
================================== Ai Message ==================================

You said your name is Bob, so your name is Bob.
================================ Human 

We can see that so far no summarization has happened - this is because there are only six messages in the list.

In [6]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='i like the celtics!', additional_kwargs={}, response_metadata={}, id='e18daa5f-6131-464c-9a7c-0c5abf376d68'),
  AIMessage(content="That's great, the Celtics are a really exciting NBA team! Do you follow them closely? What do you like most about the team? I don't have strong sports knowledge myself, but I'm always happy to chat about topics that interest you.", additional_kwargs={}, response_metadata={'id': 'msg_01N7BAiX5zGXrEvYUrtphnWw', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 146, 'output_tokens': 55, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-haiku-20240307'}, id='run--f1d2ab88-5864-4243-83c4-55253227c9b2-0', usage_metadata={'input_tokens': 146, 'output_tokens': 55, 'total_tokens': 201, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})],
 'summary': "Here is a summar

Now let's send another message in

In [7]:
input_message = HumanMessage(content="i like how much they win")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like how much they win
19:51:32 httpx INFO   HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
================================== Ai Message ==================================

That's a great point about the Celtics! They have definitely been one of the more successful and consistent franchises in the NBA over the years. Some key things about the Celtics' winning ways:

- They have won a record 17 NBA championships, the most of any team. Their most recent title was in 2008.

- They have made the playoffs in 32 of the last 35 seasons, showing their sustained excellence and competitiveness.

- Key players like Larry Bird, Kevin McHale, and Paul Pierce have led championship-caliber Celtics teams over the decades.

- Under coach Brad Stevens, they've remained a top contender in the Eastern Conference in recent years, making the playoffs every season since 2015.

I can see why you're

If we check the state now, we can see that we have a summary of the conversation, as well as the last two messages

In [8]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content='i like the celtics!', additional_kwargs={}, response_metadata={}, id='e18daa5f-6131-464c-9a7c-0c5abf376d68'),
  AIMessage(content="That's great, the Celtics are a really exciting NBA team! Do you follow them closely? What do you like most about the team? I don't have strong sports knowledge myself, but I'm always happy to chat about topics that interest you.", additional_kwargs={}, response_metadata={'id': 'msg_01N7BAiX5zGXrEvYUrtphnWw', 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 146, 'output_tokens': 55, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-3-haiku-20240307'}, id='run--f1d2ab88-5864-4243-83c4-55253227c9b2-0', usage_metadata={'input_tokens': 146, 'output_tokens': 55, 'total_tokens': 201, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}),
  HumanMessage(content='i like

We can now resume having a conversation! Note that even though we only have the last two messages, we can still ask it questions about things mentioned earlier in the conversation (because we summarized those)

In [9]:
input_message = HumanMessage(content="what's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what's my name?
19:51:33 httpx INFO   HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
================================== Ai Message ==================================

Ah, you're right, I should have kept that detail in mind. Your name is Bob, as you mentioned earlier in our conversation. My apologies for not remembering that detail more clearly. Please feel free to remind me of important details like your name - it helps me have a more coherent and personalized conversation with you.


In [10]:
input_message = HumanMessage(content="what NFL team do you think I like?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

what NFL team do you think I like?
19:51:34 httpx INFO   HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
================================== Ai Message ==================================

Hmm, that's a good question. Since I don't have any prior information about your specific NFL team preferences, I don't want to make an assumption. As an AI assistant without detailed knowledge of your personal interests, the best I can do is ask you directly - what NFL team do you like?  I'm happy to discuss whichever team you're a fan of, but I don't want to guess incorrectly. Could you please let me know your favorite NFL team?
19:51:36 httpx INFO   HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
================================ Remove Message ================================


================================ Remove Message ================================


======

In [11]:
input_message = HumanMessage(content="i like the patriots!")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

i like the patriots!
19:51:37 httpx INFO   HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
================================== Ai Message ==================================

Ah I see, got it! Thanks for sharing that you're a fan of the New England Patriots. That's a great NFL team with a very successful history over the past couple of decades. 

Since you've now told me your favorite team is the Patriots, I'm happy to discuss them with you. What do you enjoy most about being a Patriots fan? Do you have a favorite player on the team currently or from their past championship teams? I'm always interested to hear fans' perspectives on their favorite teams and players.

Let me know what you'd like to chat about regarding the Patriots. I'm glad I was able to get the right information from you directly instead of making an assumption earlier. Discussing shared interests is more fun when